In [1]:
import numpy as np
import pandas as pd
import regex as re
import emoji
import plotly.express as px
from collections import Counter
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [2]:
def startsWithDateAndTime(s):
    pattern = r"[\d]{1,2}/[\d]{1,2}/[\d]{4}" #, r"([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -" # [\d]{1,2}/[\d]{1,2}/[\d]{4} ^([0-9]+)(\/)([0-9]+)(\/)([0-9]+)
    result = re.findall(pattern, s)
    print("result :", result)
    if result:
        return True
    else:
        False

In [3]:
def FindAuthor(s):
    s=s.split(":")
    if len(s)==2:
        return True
    else:
        return False

In [4]:
def getDataPoint(line):
    splitLine = line.split(' - ')
    dateTime = splitLine[0]
    date, time = dateTime.split(', ')
    message = ' '.join(splitLine[1:])
    if FindAuthor(message):
        splitMessage = message.split(': ')
        author = splitMessage[0]
        message = ' '.join(splitMessage[1:])
    else:
        author = None
    return date, time, author, message


In [5]:
parsedData = []
conversation = "C:\\Users\\Radhika\\Downloads\\WhatsApp Chat with Chaitu.txt" # path for the whatsapp text file
with open(conversation, encoding="utf-8") as fp:

    fp.readline() # Skipping first line of the file because contains information related to something about end-to-end encryption
    print("fp.readline() : \n", fp.readline())
    messageBuffer = []
    print("messageBuffer : \n", messageBuffer)
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        print("line: \n", line)
        if startsWithDateAndTime(line):
            print("startsWithDateAndTime : \n", startsWithDateAndTime)
            if len(messageBuffer) > 0:
                parsedData.append([date, time, author, ' '.join(messageBuffer)])
                messageBuffer.clear()
                date, time, author, message = getDataPoint(line)
                messageBuffer.append(message)

        else:
            messageBuffer.append(line)

fp.readline() : 
 26/08/2023, 16:56 - Radhika: Hii aik na

messageBuffer : 
 []
line: 
 26/08/2023, 16:57 - Radhika: Te aarti chya bride to be chya veli tu te aanal hot na team bride vagere te tsl kuth bhetat
result : ['26/08/2023']
startsWithDateAndTime : 
 <function startsWithDateAndTime at 0x0000025FFCA38E50>
line: 
 26/08/2023, 16:57 - Radhika: Ani ky mhntat tyala
result : ['26/08/2023']
startsWithDateAndTime : 
 <function startsWithDateAndTime at 0x0000025FFCA38E50>
line: 
 26/08/2023, 19:22 - Chaitu: Props boltat
result : ['26/08/2023']
startsWithDateAndTime : 
 <function startsWithDateAndTime at 0x0000025FFCA38E50>
line: 
 26/08/2023, 19:22 - Chaitu: Nd te ghari bnvl hott
result : ['26/08/2023']
startsWithDateAndTime : 
 <function startsWithDateAndTime at 0x0000025FFCA38E50>
line: 
 26/08/2023, 19:22 - Chaitu: Tula online or nupur ch shejari ek shop ahe tith bhetl hadapsar mdhe
result : ['26/08/2023']
startsWithDateAndTime : 
 <function startsWithDateAndTime at 0x0000025FFCA38E5

In [6]:
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
df["Date"] = pd.to_datetime(df["Date"])
df.tail(20)

,Date,Time,Author,Message
5,2023-08-12,17:04,Chaitu,😍
6,2023-08-12,18:34,Radhika,❤️🫰🏻
7,2023-12-12,09:15,Radhika,Heyy
8,2023-12-12,09:17,Chaitu,Bol ki
9,2023-12-12,09:17,Chaitu,Gud morning
10,2023-12-12,09:18,Radhika,Good morning 🌞
11,2023-12-12,09:18,Radhika,Ky krtey
12,2023-12-12,09:18,Radhika,Lai divas zal bolan ny zal mhnun kela mag
13,2023-12-12,09:18,Radhika,Ky challay
14,2023-12-12,09:18,Chaitu,Call krych na mg


In [7]:
df.Author.unique()

array([None, 'Radhika', 'Chaitu'], dtype=object)

In [8]:
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
# print(media_messages)

def split_count(text):
    data = re.findall(r'\X', text)
    emoji_list = []

    for word in data:
        emojis = emoji.distinct_emoji_list(word)
        emoji_list.extend([emoji.demojize(is_emoji) for is_emoji in emojis])

    # emoji_list = []
    # for word in data:
    #   if any(char in emoji.distinct_emoji_list for char in word):
    #     emoji_list.append(word)

    return emoji_list

df["emoji"] = df["Message"].apply(split_count)
emojis = sum(df['emoji'].str.len())
# print(emojis)
URLPATTERN = r'(https?://\S+)'
df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
links = np.sum(df.urlcount)
# print("Data science Community")
# print("Messages:",total_messages)
print("Media:",media_messages)
print("Emojis:",emojis)
print("Links:",links)

Media: 0
Emojis: 4
Links: 1


In [9]:
media_messages_df = df[df['Message'] == '<Media omitted>']
messages_df = df.drop(media_messages_df.index)
messages_df.info()
messages_df['Letter_Count'] = messages_df['Message'].apply(lambda s : len(s))
messages_df['Word_Count'] = messages_df['Message'].apply(lambda s : len(s.split(' ')))
messages_df["MessageCount"]=1

l = ['Radhika','Chaitu'] # list of the authors name in the group

for i in range(len(l)):
    # Filtering out messages of particular user
    req_df= messages_df[messages_df["Author"] == l[i]]
    # req_df will contain messages of only one particular user
    print('\n')
    print(f'Stats of {l[i]} -')
    # shape will print number of rows which indirectly means the number of messages
    print('Messages Sent', req_df.shape[0])
    #Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
    words_per_message = (np.sum(req_df['Word_Count']))/req_df.shape[0]
    print('Words per message', words_per_message)
    #media conists of media messages
    media = media_messages_df[media_messages_df['Author'] == l[i]].shape[0]
    print('Media Messages Sent', media)
    # emojis conists of total emojis
    emojis = sum(req_df['emoji'].str.len())
    print('Emojis Sent', emojis)
    #links consist of total links
    links = sum(req_df["urlcount"])
    print('Links Sent', links)
    # print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      24 non-null     datetime64[ns]
 1   Time      24 non-null     object        
 2   Author    23 non-null     object        
 3   Message   25 non-null     object        
 4   emoji     25 non-null     object        
 5   urlcount  25 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.3+ KB


Stats of Radhika -
Messages Sent 14
Words per message 4.0
Media Messages Sent 0
Emojis Sent 3
Links Sent 0


Stats of Chaitu -
Messages Sent 9
Words per message 2.5555555555555554
Media Messages Sent 0
Emojis Sent 1
Links Sent 0


In [10]:
total_emojis_list = list([a for b in messages_df.emoji for a in b])
emoji_dict = dict(Counter(total_emojis_list))
emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)
for i in emoji_dict:
    print(i)

(':smiling_face_with_heart-eyes:', 1)
(':red_heart:', 1)
(':hand_with_index_finger_and_thumb_crossed_light_skin_tone:', 1)
(':sun_with_face:', 1)


In [20]:
text = " ".join(review for review in messages_df.Message)
print("There are {} words in all the messages.".format(len(text)))
stopwords = set(STOPWORDS)

# Generate a word cloud image with specific font_path
#wordcloud = WordCloud(stopwords=stopwords, background_color="white", font_path='‪‪C:\Windows\Fonts\GARABD.TTF').generate(text)
#wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)
wordcloud = WordCloud(width=800, height=400, background_color="black").generate(text)

# Display the generated image using Matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
#plt.axis("off")
plt.show()

There are 493 words in all the messages.


ValueError: Only supported for TrueType fonts

In [ ]:
l = ['Radhika','Chaitu']
for i in range(len(l)):
    dummy_df = messages_df[messages_df['Author'] == l[i]]
    text = " ".join(review for review in dummy_df.Message)
    stopwords = set(STOPWORDS)
    #Generate a word cloud image
    print('Author name',l[i])
    wordcloud = WordCloud(stopwords=stopwords, interpolation='bilinear').generate(text)
    #Display the generated image
    plt.figure( figsize=(10,5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [13]:
import pandas as pd
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

# Download NLTK resources
nltk.download('vader_lexicon')
nltk.download('stopwords')

# Load chat data (replace 'your_chat_data.csv' with your actual file)
chat_data = "C:\\Users\\Radhika\\Downloads\\WhatsApp Chat with Chaitu.txt" 

# Assuming your chat data has a 'text' column
text_data = chat_data['text'].astype(str)

# Preprocessing the text data
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = ''.join([char for char in text if char.isalpha() or char.isspace()])
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply preprocessing to each text in the dataset
text_data_processed = text_data.apply(preprocess_text)

# Combine processed text into a single string
combined_text = ' '.join(text_data_processed)

# Create WordCloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(combined_text)

# Display WordCloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Word Cloud of Chat Data")
plt.show()

# Save WordCloud image
wordcloud.to_file("wordcloud_chat_data.png")

# Sentiment Analysis using NLTK's SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Example: Analyzing sentiment of a single sentence
example_sentence = "I love this product! It's amazing."
sentiment_scores = sia.polarity_scores(example_sentence)
print(f"Sentiment Scores: {sentiment_scores}")

# Analyzing sentiment of the entire chat data
sentiments = text_data.apply(lambda x: sia.polarity_scores(x)['compound'])
chat_data['sentiment'] = sentiments

# Display the first few rows of the chat data with sentiment scores
print(chat_data[['text', 'sentiment']].head())


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Radhika\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Radhika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: string indices must be integers

In [19]:
import re
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from textblob import TextBlob

# Load the chat data from the text file
conversation_path = "C:\\Users\\Radhika\\Downloads\\WhatsApp Chat with Chaitu.txt"
with open(conversation_path, 'r', encoding='utf-8') as file:
    raw_text = file.read()

# Extracting messages and timestamps using regular expressions
pattern = re.compile(r'(\d{1,2}/\d{1,2}/\d{2,4},? \d{1,2}:\d{2} [APMapm]{2} - .+?): (.+)')
matches = pattern.findall(raw_text)
data = {'timestamp': [], 'message': []}
for match in matches:
    timestamp, message = match
    data['timestamp'].append(timestamp)
    data['message'].append(message)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Preprocess the text data
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['clean_message'] = df['message'].apply(preprocess_text)

# Create Word Cloud
text_for_wordcloud = ' '.join(df['clean_message'])
wordcloud = WordCloud(width=800, height=400, background_color="black").generate(text_for_wordcloud)
print(wordcloud)
# Display the Word Cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Save the Word Cloud image
wordcloud.to_file("wordcloud.png")

# Perform Sentiment Analysis using TextBlob
df['sentiment'] = df['clean_message'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Display Sentiment Distribution
plt.figure(figsize=(10, 5))
df['sentiment'].plot(kind='hist', bins=20, edgecolor='black')
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment Polarity')
plt.ylabel('Frequency')
plt.show()


ValueError: We need at least 1 word to plot a word cloud, got 0.